# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [23]:
import pandas as pd
import pickle
from sqlalchemy import create_engine
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# load data from database
engine = create_engine('sqlite:///messages.db')
df = pd.read_sql('messages_table', engine)
messages = df.iloc[:,0:3]
categories = df.iloc[:,4:-1] 

In [3]:
categories = categories.astype(int)

In [4]:
categories.sum()

related                   20282
request                    4474
offer                       118
aid_related               10860
medical_help               2084
medical_products           1313
search_and_rescue           724
security                    471
military                    860
child_alone                   0
water                      1672
food                       2923
shelter                    2314
clothing                    405
money                       604
missing_people              298
refugees                    875
death                      1194
other_aid                  3446
infrastructure_related     1705
transport                  1201
buildings                  1333
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7297
floods                     2155
storm                      2443
fire    

In [5]:
#There are no features of this type, therefore dropping
categories.drop(columns=['child_alone'], inplace=True)

### 2. Write a tokenization function to process your text data

In [6]:
# We'll need the tokenizer and the vocab in order to re-use the model.
# I'm using Keras' features here because they're incredibly quick and take take of the casing 
# and punctuation already.
# It also supports sequencing (i.e. Word2Vec) and matrices so we can play around with the 
# text input models.
# See https://keras.io/preprocessing/text/ for details


def tokenize(text, vocab_size):
    tokenizer = Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(text)
    encoded_docs = tokenizer.texts_to_matrix(text, mode = 'count')
    return tokenizer, encoded_docs

In [7]:
messages.head()

,id,message,original
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ..."
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...


In [8]:
tok, X = tokenize(messages['message'], 5000)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
classifier = MultiOutputClassifier(RandomForestClassifier(n_jobs=4))
pipeline = Pipeline([
    ('classifier', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, categories, test_size=0.2, random_state=42)

In [11]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('classifier', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
y_pred = pipeline.predict(X_test)

In [13]:
from sklearn.metrics import classification_report
for i in range (0, len(y_pred[0])-1):
    truth = y_test.iloc[:,i:i+1]
    prediction = y_pred[:,i:i+1]
    print(y_test.columns[i], "\n", classification_report(truth, prediction))

related 
              precision    recall  f1-score   support

          0       0.60      0.38      0.47      1266
          1       0.82      0.92      0.86      3938
          2       0.54      0.50      0.52        40

avg / total       0.76      0.78      0.77      5244

request 
              precision    recall  f1-score   support

          0       0.89      0.98      0.93      4349
          1       0.81      0.41      0.54       895

avg / total       0.88      0.88      0.87      5244

offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

aid_related 
              precision    recall  f1-score   support

          0       0.75      0.86      0.80      3113
          1       0.74      0.59      0.66      2131

avg / total       0.75      0.75      0.74      5244

medical_help 
              precision    reca

/Users/carl.sharpe/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
parameters = [{'classifier__estimator__n_estimators': [20, 30],
              'classifier__estimator__criterion': ['gini', 'entropy']}]
cv = GridSearchCV(pipeline, parameters, cv=5)

In [15]:
cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('classifier', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,...ob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'classifier__estimator__n_estimators': [20, 30], 'classifier__estimator__criterion': ['gini', 'entropy']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
means = cv.cv_results_['mean_test_score']
stds = cv.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, cv.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

0.264 (+/-0.010) for {'classifier__estimator__criterion': 'gini', 'classifier__estimator__n_estimators': 20}
0.265 (+/-0.016) for {'classifier__estimator__criterion': 'gini', 'classifier__estimator__n_estimators': 30}
0.255 (+/-0.010) for {'classifier__estimator__criterion': 'entropy', 'classifier__estimator__n_estimators': 20}
0.262 (+/-0.015) for {'classifier__estimator__criterion': 'entropy', 'classifier__estimator__n_estimators': 30}



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
classifier = MultiOutputClassifier(
    RandomForestClassifier(n_estimators=1000, n_jobs=4))
pipeline = Pipeline([
    ('classifier', MultiOutputClassifier(RandomForestClassifier()))
])

In [20]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('classifier', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [21]:
y_pred = pipeline.predict(X_test)

In [22]:
from sklearn.metrics import classification_report
for i in range (0, len(y_pred[0])-1):
    truth = y_test.iloc[:,i:i+1]
    prediction = y_pred[:,i:i+1]
    print(y_test.columns[i], "\n", classification_report(truth, prediction))

related 
              precision    recall  f1-score   support

          0       0.61      0.37      0.46      1266
          1       0.82      0.92      0.87      3938
          2       0.56      0.62      0.59        40

avg / total       0.77      0.79      0.77      5244

request 
              precision    recall  f1-score   support

          0       0.89      0.98      0.93      4349
          1       0.83      0.41      0.55       895

avg / total       0.88      0.88      0.87      5244

offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

aid_related 
              precision    recall  f1-score   support

          0       0.75      0.87      0.81      3113
          1       0.75      0.58      0.66      2131

avg / total       0.75      0.75      0.74      5244

medical_help 
              precision    reca

/Users/carl.sharpe/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [26]:
tokenizer_filename = "tokenizer.pickle"
classifier_filename = "classifier.pickle"

tokenizer_file = open(tokenizer_filename, 'wb')
pickle.dump(tok, tokenizer_file)

classifier_file = open(classifier_filename, 'wb')
pickle.dump(pipeline, classifier_file)



### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.